In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# Part 1 create Toronto neighborhood dataframe

## 1) The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [32]:
soup_html = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text, 'html.parser')

rows = []
for row in soup_html.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) == 3):
        rows.append([cells[0].text, cells[1].text, cells[2].text.rstrip('\n')])
df = pd.DataFrame(rows, columns=["PostalCode", "Borough", "Neighborhood"])

In [33]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
## 3) More than one neighborhood can exist in one postal code area.

In [27]:
df_dropna = df[df.Borough != "Not assigned"].reset_index(drop=True)
df_grouped_borough = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))


In [28]:
df_grouped_borough.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 4) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [29]:
df_grouped_borough.loc[df['Neighborhood'] == 'Not assigned','Neighborhood'] = df_grouped_borough['Borough']


In [30]:
df_grouped_borough.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Scarborough
1,M1C,Scarborough,Scarborough
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [31]:
df_grouped_borough.shape

(103, 3)

# Part 2 get the latitude and the longitude coordinates of each neighborhood

In [11]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
df_grouped_borough_coord = df_grouped_borough.merge(coordinates, on="PostalCode", how="left")


In [12]:
df_grouped_borough_coord

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


# Part 3 Explore and cluster the neighborhoods in Toronto

## I decided to find the Neighborhood with most chinese Food

In [190]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [191]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [192]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_grouped_borough_coord['Latitude'], df_grouped_borough_coord['Longitude'], df_grouped_borough_coord['Borough'], df_grouped_borough_coord['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
   
map_toronto

In [195]:
CLIENT_ID = 'key' # your Foursquare ID
CLIENT_SECRET = 'key' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100

In [196]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [197]:
toronto_venues = getNearbyVenues(names=df_grouped_borough_coord['Neighborhood'],
                                   latitudes=df_grouped_borough_coord['Latitude'],
                                   longitudes=df_grouped_borough_coord['Longitude'],
                                radius=4000)

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [198]:
print(toronto_venues.shape)
toronto_venues.head(10)

(10194, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
1,"Rouge, Malvern",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
2,"Rouge, Malvern",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
3,"Rouge, Malvern",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
4,"Rouge, Malvern",43.806686,-79.194353,Penguin Exhibit,43.819435,-79.185959,Zoo Exhibit
5,"Rouge, Malvern",43.806686,-79.194353,Orangutan Exhibit,43.818413,-79.182548,Zoo Exhibit
6,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
7,"Rouge, Malvern",43.806686,-79.194353,Australasia Pavillion,43.822563,-79.183286,Zoo Exhibit
8,"Rouge, Malvern",43.806686,-79.194353,Morningside Park,43.786546,-79.205322,Park
9,"Rouge, Malvern",43.806686,-79.194353,"Lamanna's Bakery, Cafe & Fine Foods",43.797971,-79.148432,Bakery


In [199]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",100,100,100,100,100,100
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",100,100,100,100,100,100
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Downsview North, Wilson Heights",100,100,100,100,100,100
Bayview Village,100,100,100,100,100,100
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100


In [200]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


In [201]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Liquor Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,1,0,0,0,0,0,0,0,0,0,0,0

In [202]:
toronto_onehot.shape

(10194, 276)

In [203]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo Exhibit,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Liquor Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,"Adelaide, King, Richmo

In [205]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.05
1                Hotel  0.05
2   Italian Restaurant  0.04
3  Japanese Restaurant  0.03
4              Theater  0.03


----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.09
1           Coffee Shop  0.06
2  Caribbean Restaurant  0.05
3                Bakery  0.04
4     Indian Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                   venue  freq
0     Chinese Restaurant  0.15
1                 Bakery  0.06
2        Bubble Tea Shop  0.05
3  Vietnamese Restaurant  0.04
4            Coffee Shop  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Coffee Shop  0.17
1  Fast Food Restaurant  0.09
2     Indian Restaurant  0.06
3        Sandwich Place  0.06
4           Pizza Place  0.05


----Alderwood, Long Branch----
       

                venue  freq
0                Park  0.12
1  Italian Restaurant  0.08
2         Coffee Shop  0.05
3                Café  0.03
4           Bookstore  0.03


----Newtonbrook, Willowdale----
                       venue  freq
0          Korean Restaurant  0.12
1                Coffee Shop  0.06
2  Middle Eastern Restaurant  0.06
3                       Café  0.04
4                Supermarket  0.03


----North Toronto West----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.07
2                Café  0.06
3                Park  0.05
4              Bakery  0.05


----Northwest----
                  venue  freq
0           Coffee Shop  0.15
1                 Hotel  0.09
2            Steakhouse  0.05
3        Airport Lounge  0.05
4  Fast Food Restaurant  0.05


----Northwood Park, York University----
            venue  freq
0     Coffee Shop  0.13
1      Restaurant  0.05
2  Sandwich Place  0.05
3     Pizza Place  0.04
4        Pharmacy  0.04


---

In [206]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [207]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Hotel,Coffee Shop,Italian Restaurant,Theater,Gym,Restaurant,Café,Sandwich Place,Steakhouse,Japanese Restaurant
1,Agincourt,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Supermarket,Bakery,Clothing Store,Bubble Tea Shop,Restaurant,Pharmacy
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Bakery,Bubble Tea Shop,Indian Restaurant,Vietnamese Restaurant,Coffee Shop,Caribbean Restaurant,Dessert Shop,Supermarket,Noodle House
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Pizza Place,Bank,Gas Station,Grocery Store,Pharmacy,Asian Restaurant
4,"Alderwood, Long Branch",Coffee Shop,Burger Joint,Bakery,Grocery Store,Liquor Store,Seafood Restaurant,Burrito Place,Café,Pizza Place,Middle Eastern Restaurant


In [208]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


In [209]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [210]:

toronto_merged = df_grouped_borough_coord

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4,Zoo Exhibit,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Gas Station,Pizza Place,Breakfast Spot,Burger Joint,Fried Chicken Joint
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Coffee Shop,Pizza Place,Park,Fast Food Restaurant,Sandwich Place,Pharmacy,Breakfast Spot,Burger Joint,Grocery Store,Supermarket
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Coffee Shop,Pizza Place,Sandwich Place,Park,Fast Food Restaurant,Burger Joint,Pharmacy,Discount Store,Fried Chicken Joint,Indian Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Indian Restaurant,Clothing Store,Pharmacy,Pizza Place,Caribbean Restaurant,Gas Station,Park,Gym,Fried Chicken Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Coffee Shop,Indian Restaurant,Park,Breakfast Spot,Sandwich Place,Steakhouse,Caribbean Restaurant,Train Station,Asian Restaurant,Fast Food Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Coffee Shop,Pharmacy,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Bank,Chinese Restaurant,Park,Burger Joint,Clothing Store
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Burger Joint,Coffee Shop,Middle Eastern Restaurant,Park,Chinese Restaurant,Indian Restaurant,Supermarket,Pharmacy,Liquor Store,Fast Food Restaurant
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Middle Eastern Restaurant,Coffee Shop,Burger Joint,Bakery,Supermarket,Bank,Park,Fish & Chips Shop,Indian Restaurant,Burrito Place
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,4,Coffee Shop,Fast Food Restaurant,Park,Chinese Restaurant,Bank,Pizza Place,Sandwich Place,Beach,Burger Joint,Beer Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,Park,Coffee Shop,Beach,Breakfast Spot,Thai Restaurant,Bar,Pub,Japanese Restaurant,Mexican Restaurant,Fish & Chips Shop


In [211]:
import math
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
        
map_clusters

## Examine Clusters

### Cluster 1

In [212]:
cluster_label = 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Coffee Shop,Indian Restaurant,Park,Breakfast Spot,Sandwich Place,Steakhouse,Caribbean Restaurant,Train Station,Asian Restaurant,Fast Food Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Burger Joint,Coffee Shop,Middle Eastern Restaurant,Park,Chinese Restaurant,Indian Restaurant,Supermarket,Pharmacy,Liquor Store,Fast Food Restaurant
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Middle Eastern Restaurant,Coffee Shop,Burger Joint,Bakery,Supermarket,Bank,Park,Fish & Chips Shop,Indian Restaurant,Burrito Place
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,Park,Coffee Shop,Beach,Breakfast Spot,Thai Restaurant,Bar,Pub,Japanese Restaurant,Mexican Restaurant,Fish & Chips Shop
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,0,Coffee Shop,Middle Eastern Restaurant,Supermarket,Burger Joint,Gym,Pizza Place,Clothing Store,Indian Restaurant,Park,Breakfast Spot


In [213]:
cluster_label = 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,M4C,East York,Woodbine Heights,43.695344,-79.318389,1,Brewery,Café,Park,Gastropub,Coffee Shop,Greek Restaurant,Indian Restaurant,American Restaurant,Bakery,BBQ Joint
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Park,Beach,Coffee Shop,Gastropub,Brewery,Café,Breakfast Spot,Pub,American Restaurant,Bakery
38,M4G,East York,Leaside,43.709060,-79.363452,1,Park,Italian Restaurant,Coffee Shop,Grocery Store,Bakery,Café,Gym,Brewery,Liquor Store,Supermarket
39,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1,Park,Café,Greek Restaurant,Grocery Store,Gastropub,Bakery,Ice Cream Shop,Gym,Brewery,Coffee Shop
40,M4J,East York,East Toronto,43.685347,-79.338106,1,Café,Brewery,Greek Restaurant,Park,Coffee Shop,Farmers Market,Gastropub,American Restaurant,Ice Cream Shop,Asian Restaurant


In [214]:
cluster_label = 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Coffee Shop,Park,Japanese Restaurant,Café,Restaurant,Italian Restaurant,Hotel,Diner,Theater,Bakery
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,2,Coffee Shop,Park,Café,Italian Restaurant,Japanese Restaurant,Farmers Market,Hotel,Diner,Historic Site,Middle Eastern Restaurant
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Park,Japanese Restaurant,Café,Restaurant,Italian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Theater,Steakhouse
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Café,Park,Japanese Restaurant,Hotel,Gastropub,Restaurant,Farmers Market,Steakhouse,Italian Restaurant
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Coffee Shop,Hotel,Japanese Restaurant,Restaurant,Park,Farmers Market,Theater,Italian Restaurant,Café,Steakhouse


In [215]:
cluster_label = 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,3,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Supermarket,Bakery,Clothing Store,Bubble Tea Shop,Restaurant,Pharmacy
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,3,Chinese Restaurant,Coffee Shop,Bakery,Caribbean Restaurant,Noodle House,Supermarket,Breakfast Spot,Pharmacy,Middle Eastern Restaurant,Toy / Game Store
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,3,Chinese Restaurant,Bakery,Bubble Tea Shop,Indian Restaurant,Vietnamese Restaurant,Coffee Shop,Caribbean Restaurant,Dessert Shop,Supermarket,Noodle House
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,3,Chinese Restaurant,Bakery,Japanese Restaurant,Caribbean Restaurant,Asian Restaurant,Coffee Shop,Bubble Tea Shop,Noodle House,Sushi Restaurant,Hotel
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,Chinese Restaurant,Bakery,Japanese Restaurant,Caribbean Restaurant,Breakfast Spot,Pharmacy,Coffee Shop,Asian Restaurant,Sporting Goods Shop,Electronics Store


In [216]:
cluster_label = 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_label].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4,Zoo Exhibit,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Gas Station,Pizza Place,Breakfast Spot,Burger Joint,Fried Chicken Joint
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Coffee Shop,Pizza Place,Park,Fast Food Restaurant,Sandwich Place,Pharmacy,Breakfast Spot,Burger Joint,Grocery Store,Supermarket
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Coffee Shop,Pizza Place,Sandwich Place,Park,Fast Food Restaurant,Burger Joint,Pharmacy,Discount Store,Fried Chicken Joint,Indian Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Indian Restaurant,Clothing Store,Pharmacy,Pizza Place,Caribbean Restaurant,Gas Station,Park,Gym,Fried Chicken Joint
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Coffee Shop,Pharmacy,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Bank,Chinese Restaurant,Park,Burger Joint,Clothing Store
